In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from collections import Counter
import re

df = pd.read_csv('Allstatesinsurvey/all_deaths.csv', encoding='cp1252')

In [2]:
# cleaning (lowercase)

df_temp = df
df_temp['gender'] = df_temp['gender'].str.lower()
df_temp['cause_short'] = df_temp['cause_short'].str.lower()
df_temp['what_info_from_media'] = df_temp['what_info_from_media'].str.lower()
df_temp['num_media_descriptors'] = 0
df_temp['num_media_descriptors'] = df_temp['what_info_from_media'].str.split(",").str.len()

df_temp['cause_detail'] = df_temp['cause_detail'].str.lower()
df_temp['cause_detail'] = df_temp['cause_detail'].where(df_temp['cause_detail']!='unknown')

print(df_temp.columns)

Index(['id', 'state', 'county', 'jail', 'year', 'date_of_death', 'full_name',
       'last_name', 'first_name', 'mid_name', 'suffix', 'date_incarcerated',
       'cause_short', 'cause_detail', 'dob', 'yob', 'age', 'race',
       'race_detail', 'gender', 'custody_status', 'what_info_from_media',
       'num_media_descriptors'],
      dtype='object')


In [3]:
# date-time
df_temp['date_of_death'] = pd.to_datetime(arg=df_temp['date_of_death'], errors='coerce', format='%m/%d/%Y')
df_temp['dob'] = pd.to_datetime(arg=df_temp['dob'], errors='coerce', format='%m/%d/%Y')
df_temp['date_incarcerated'] = pd.to_datetime(arg=df_temp['date_incarcerated'], errors='coerce', format='%m/%d/%Y')
df_temp['length_incarceration'] = (df_temp['date_of_death'] - df_temp['date_incarcerated']).dt.days

Using regex to standardize detailed death information to better understand poorly-categorized or un-categorized (in `cause_short`) death information.

In [6]:
df_regex_temp = df_temp

cancer_regex = "cancer|oncology|carcinoma|metastatic"
med_no_cancer_regex = "medical|natural|heart attack|seizure|illness|natural|disease|blood pressure|blood clot|sepsis|bowel|cirrhosis|cardio|pulm|cardiac|diab|aids|organ|ulcer"
no_details_regex = "unknown|undetermined|unspecified|cannot be determined"
hanging_suffocation_suicide_regex = "hanging|hanged|strangle|strangulation|suffocation|not breathing|jump|cutting|cut|suicide"
drug_regex = "overdose|drug|withdrawal|withdraw|needle|injection|toxic|narcotic|alcohol|morphine|hydrocodone"

regex_categories = {
    "cancer" : cancer_regex, # “cancer”, “oncology”, “carcinoma”, “metastatic”
    "medical_no_cancer" : med_no_cancer_regex, # “medical”, “natural”, “heart attack”, “seizure”, “illness”, “natural”, “disease”
    "hanging/suffocation/suicide" : hanging_suffocation_suicide_regex, # “hanging”, “hanged”, ”strangulation”, “suffocation”, “not breathing”, “jump”, “cutting”, “cut”"
    "drug" : drug_regex, # “overdose”, “drug”, “withdrawal”, “needle”, “injection”
    "no_details" : no_details_regex # “unknown”, “undetermined”, “unspecified”, “cannot be determined” 
}

no_category = []

def get_regex_cat(str):
    if not bool(str) or pd.isnull(str):
        return np.nan
    for cat in regex_categories.keys():
        if re.search(regex_categories[cat], str):
            return cat
    no_category.extend(str.split())
    return "NO CATEGORY FOUND (NON-EMPTY)"

c = Counter(no_category)
top_10 = c.most_common(10)

df_regex_temp["cause_detail_group"] = df_regex_temp.apply(lambda row: get_regex_cat(row.cause_detail), axis=1)

In [7]:
# save
df_temp.to_csv('all_deaths_clean.csv', encoding='utf-8', index=False)